# Analysis of cross-validation results

In [1]:
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold

from mpstool.cv_metrics import brier_score, zero_one_score, balanced_linear_score

from geone.img import readImageGslib, readPointSetGslib
from geone.img import Img
from geone.imgplot import drawImage2D
from geone.deesseinterface import DeesseEstimator

In [2]:
OUTPUT_DIR = 'output/'
DATA_DIR = 'data_roussillon/'
SAMPLES_DIR = DATA_DIR

## Training image selection

The benchmark case of training image selection with three candidate training images.
First, we check sensitivity to number of realisations for probabilities estimation.
Second, training image selection for different data sets.

## Parameter selection (Roussillon)

Three datasets with: 50, 150, 600 points each. In output directory: roussillon_observations_50.csv, etc.

In [10]:
df_roussillon = pd.read_csv('df_roussillon.csv', index_col=0)
df_roussillon.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_distanceThreshold,param_maxScanFraction,param_nneighboringNode,params,split0_test_brier,split1_test_brier,...,std_test_brier,rank_test_brier,split0_test_skill_brier,split1_test_skill_brier,split2_test_skill_brier,split3_test_skill_brier,split4_test_skill_brier,mean_test_skill_brier,std_test_skill_brier,rank_test_skill_brier
0,0.000977,0.000060,133.192147,0.964267,"[0.12501, 0.12501]",0.1,"[8, 1]","{'distanceThreshold': [0.12501, 0.12501], 'max...",-0.374104,-0.407677,...,0.046722,38,0.211764,0.141026,0.338754,0.407659,0.374480,0.294737,0.101548,38
1,0.000979,0.000071,126.318612,0.643584,"[0.12501, 0.12501]",0.2,"[8, 1]","{'distanceThreshold': [0.12501, 0.12501], 'max...",-0.391010,-0.363833,...,0.044353,29,0.176143,0.233405,0.378831,0.387590,0.423203,0.319834,0.096823,30
2,0.000940,0.000070,121.645797,0.658246,"[0.12501, 0.12501]",0.4,"[8, 1]","{'distanceThreshold': [0.12501, 0.12501], 'max...",-0.389406,-0.351958,...,0.040956,27,0.179523,0.258425,0.359912,0.398224,0.415093,0.322235,0.089727,27
3,0.000943,0.000074,118.091318,1.159128,"[0.12501, 0.12501]",0.8,"[8, 1]","{'distanceThreshold': [0.12501, 0.12501], 'max...",-0.397854,-0.379156,...,0.036661,44,0.161723,0.201119,0.251380,0.349391,0.371991,0.267121,0.081828,44
4,0.000977,0.000060,98.699392,0.640770,"[0.25001, 0.25001]",0.1,"[8, 1]","{'distanceThreshold': [0.25001, 0.25001], 'max...",-0.348479,-0.370365,...,0.030353,28,0.265756,0.219643,0.354118,0.384430,0.385980,0.321985,0.067324,28


In [13]:
df_sorted = df_roussillon.sort_values('rank_test_brier', ascending=True)
df_sorted[['mean_score_time', 'param_distanceThreshold', 'param_maxScanFraction', 'param_nneighboringNode', 'mean_test_brier', 'std_test_brier']].head(10)

,mean_score_time,param_distanceThreshold,param_maxScanFraction,param_nneighboringNode,mean_test_brier,std_test_brier
13,125.135196,"[0.18751, 0.18751]",0.2,"[16, 1]",-0.284419,0.030108
14,119.810546,"[0.18751, 0.18751]",0.4,"[16, 1]",-0.287654,0.035224
28,151.802568,"[0.18751, 0.18751]",0.1,"[32, 1]",-0.290167,0.037982
32,135.149330,"[0.25001, 0.25001]",0.1,"[32, 1]",-0.293104,0.041715
15,115.737523,"[0.18751, 0.18751]",0.8,"[16, 1]",-0.293235,0.033259
12,131.534910,"[0.18751, 0.18751]",0.1,"[16, 1]",-0.294612,0.044701
17,107.457373,"[0.25001, 0.25001]",0.2,"[16, 1]",-0.296737,0.035285
29,145.378928,"[0.18751, 0.18751]",0.2,"[32, 1]",-0.297075,0.035513
16,112.460010,"[0.25001, 0.25001]",0.1,"[16, 1]",-0.297406,0.047480
34,123.283724,"[0.25001, 0.25001]",0.4,"[32, 1]",-0.298627,0.035543


In [14]:
df_roussillon.mean_score_time.sum()*5/3600

12.00817327260971

In [15]:
df_dsbc_roussillon = pd.read_csv('df_dsbc_roussillon.csv', index_col=0)

In [16]:
df_dsbc_roussillon.mean_score_time.sum()*5/3600

15.534730938739246

In [17]:
df_sorted = df_dsbc_roussillon.sort_values('rank_test_brier', ascending=True)
df_sorted[['mean_score_time', 'param_distanceThreshold', 'param_maxScanFraction', 'param_nneighboringNode', 'mean_test_brier', 'std_test_brier']].head(10)

,mean_score_time,param_distanceThreshold,param_maxScanFraction,param_nneighboringNode,mean_test_brier,std_test_brier
27,185.341210,"[1e-05, 1e-05]",0.04,"[8, 1]",-0.343154,0.036394
34,188.499041,"[1e-05, 1e-05]",0.08,"[16, 1]",-0.344015,0.046446
35,186.057693,"[1e-05, 1e-05]",0.08,"[8, 1]",-0.344404,0.036407
26,185.962951,"[1e-05, 1e-05]",0.04,"[16, 1]",-0.346050,0.039948
42,179.722090,"[1e-05, 1e-05]",0.20,"[16, 1]",-0.347342,0.041977
30,188.544875,"[1e-05, 1e-05]",0.06,"[16, 1]",-0.347844,0.044698
31,190.846252,"[1e-05, 1e-05]",0.06,"[8, 1]",-0.348271,0.038410
39,182.546415,"[1e-05, 1e-05]",0.10,"[8, 1]",-0.348725,0.031477
41,179.241866,"[1e-05, 1e-05]",0.20,"[32, 1]",-0.348731,0.044183
38,183.511453,"[1e-05, 1e-05]",0.10,"[16, 1]",-0.349690,0.043036


In [ ]:
def best_results_for_each_TI(nsamples, score, score_name):
    df = pd.read_csv(OUTPUT_DIR+'roussillon_observations_{}.csv'.format(nsamples))
    info = ['param_TI',
        'param_distanceThreshold',
        'param_maxScanFraction',
        'param_nneighboringNode',
        ]
    ref = reference_score(observation_filename=SAMPLES_DIR+'roussillon_observations_{}.gslib'.format(nsamples),
                          score=score, varname='Facies_real00000')
    df['ref_score'] = ref
    df1 = df[df['param_TI'] == 'data/trueTI.gslib'].sort_values('mean_test_'+score_name,ascending=False).head(1)
    df2 = df[df['param_TI'] == 'data/analogTI.gslib'].sort_values('mean_test_'+score_name,ascending=False).head(1)
    return df1.append(df2, ignore_index=True)

In [ ]:
info = ['param_TI',
        'param_distanceThreshold',
        'param_maxScanFraction',
        'param_nneighboringNode',
        'mean_test_score',
        'score_method',
        'nsamples',
        'ref_score'
       ]

df_best_roussillon = pd.DataFrame()
for nsamples in [50, 150, 600]:
    for score in [(brier_score, 'brier'), (zero_one_score, 'zero_one'), (balanced_linear_score, 'linear')]:
        df = best_results_for_each_TI(nsamples, score[0], score[1])
        df['mean_test_score'] = df['mean_test_{}'.format(score[1])]
        df['score_method'] = score[1]
        df['nsamples'] = nsamples
        df_best_roussillon = df_best_roussillon.append(df[info],ignore_index=True)
df_best_roussillon  

In [ ]:
new_columns = {
    "param_TI" : "TI",
    "param_distanceThreshold" : "t",
    "param_maxScanFraction" : "f",
    "param_nneighboringNode" : "n",
    "mean_test_score" : "score",
    "ref_score" : "reference",   
    "score_method" : "function",
    "nsamples" : "wells",
}

df_renamed = df_best_roussillon.rename(columns=new_columns)
df_renamed.head()

In [ ]:
def transform_entries(df_original):
    df = df_original.copy()
    df['TI'] = df['TI'].apply(lambda x: x.split('/')[1].split('TI')[0])
    df['t'] = df['t'].apply(lambda x: eval(x)[0])
    df['n'] = df['n'].apply(lambda x: eval(x)[0])
    df['score'] = df['score'].round(2)
    df['reference'] = df['reference'].round(2)
    df['function'] = df['function'].apply(lambda x: x.replace('_', '-'))
    return df

df_publication = transform_entries(df_renamed)
df_publication.head()

In [ ]:
df_latex = df_publication[['wells', 'function', 'TI', 'score', 'reference', 't', 'f', 'n']].to_latex('tables/table-roussillon.tex', index=False)


# Figures for publication

In [ ]:
COLOR_SCHEME = [ 
        [x/255 for x in [166,206,227]],  
        [x/255 for x in [31,120,180]],
        [x/255 for x in [51,160,44]],
      ]
FONTSIZE = 16
FIG_DIR = 'figures/'
DPI=600

import matplotlib

matplotlib.rcParams["image.interpolation"] = None
matplotlib.rcParams['pdf.fonttype'] = 42

In [ ]:
def sensitivity_plot(test_case, title, ylabel, scoring, filename, score, ylim=[-1.0, 0.0], loc='lower right'):
    fig = plt.figure(figsize=(5,5))
    ax = plt.axes()
    df = sensitivity_results[test_case]
    colors=COLOR_SCHEME
    labels=['TI: A', 'TI: B', 'TI: C', 'ref.']
    markers=['o', 'x', '*']

    for i, ti in enumerate(['A', 'B', 'C']):
        df_plot = df[df['param_TI'] == 'data/{}.gslib'.format(ti)]
        ax.plot(df_plot['param_nrealization'], df_plot[scoring], label=labels[i], linestyle='--', color=colors[i], marker=markers[i])
    # reference line
    x = np.linspace(0,50)
    ax.plot(x, np.ones(len(x))*reference_score(SAMPLES_DIR+'sample_{}_50.gslib'.format(test_case), score), linestyle='--', color='black', label=labels[-1])
    
    ax.legend(loc=loc, fontsize=FONTSIZE, ncol=2)
    ax.set_xlabel("#realizations", fontsize=FONTSIZE)
    ax.set_ylim(ylim)
    ax.set_ylabel(" ", fontsize=FONTSIZE)  
    ax.set_title("{}, {}".format(ylabel,title), fontsize=FONTSIZE)
    ax.tick_params(axis='both', which='major', labelsize=FONTSIZE)
    plt.savefig(filename, dpi=DPI, bbox_inches='tight')
    #!convert -trim $filename $filename
    !pdfcrop $filename $filename

In [ ]:
sensitivity_plot('A', 'a)','mean quadratic score', 'mean_test_brier', FIG_DIR+'sensitivity_A_brier.pdf', brier_score)

In [ ]:
sensitivity_plot('B', 'b)','mean quadratic score', 'mean_test_brier', FIG_DIR+'sensitivity_B_brier.pdf', brier_score)

In [ ]:
sensitivity_plot('C', 'c)','mean quadratic score', 'mean_test_brier', FIG_DIR+'sensitivity_C_brier.pdf', brier_score)

In [ ]:
sensitivity_plot('A', 'a)','mean zero-one score', 'mean_test_zero_one', FIG_DIR+'sensitivity_A_zero_one.pdf', score=zero_one_score, ylim=[0.5,1], loc=None)

In [ ]:
sensitivity_plot('B', 'b)','mean zero-one score', 'mean_test_zero_one', FIG_DIR+'sensitivity_B_zero_one.pdf', score=zero_one_score, ylim=[0.5,1], loc='upper right')

In [ ]:
sensitivity_plot('C', 'c)','mean zero-one score', 'mean_test_zero_one', FIG_DIR+'sensitivity_C_zero_one.pdf', score=zero_one_score, ylim=[0.5,1], loc=(0.2,0.6))

In [ ]:
sensitivity_plot('A', 'a)','mean balanced linear score', 'mean_test_linear', FIG_DIR+'sensitivity_A_linear.pdf', score=balanced_linear_score, ylim=[0.3,1], loc="upper right")

In [ ]:
sensitivity_plot('B', 'b)','mean balanced linear score', 'mean_test_linear', FIG_DIR+'sensitivity_B_linear.pdf', score=balanced_linear_score, ylim=[0.3,1])

In [ ]:
sensitivity_plot('C', 'c)','mean balanced linear score', 'mean_test_linear', FIG_DIR+'sensitivity_C_linear.pdf', score=balanced_linear_score, ylim=[0.3,1])

## TI selection

In [ ]:
def ti_selection_plot(test_case, title, ylabel, scoring, filename, score_ref, ylim=[-1,0], loc="lower right"):
    fig = plt.figure(figsize=(5,5))
    ax = plt.axes()
    df = df_ti_selection[df_ti_selection.type==test_case].sort_values(by = 'nsamples')
    colors=COLOR_SCHEME
    labels=['TI: A', 'TI: B', 'TI: C', 'ref']
    markers=['o', 'x', '*']
    for i, ti in enumerate(['A', 'B', 'C']):
        df_plot = df[df['param_TI'] == 'data/{}.gslib'.format(ti)]
        ax.semilogx(df_plot['nsamples'], df_plot[scoring], label=labels[i], linestyle='--', color=colors[i], marker=markers[i])
    ax.semilogx(df_plot['nsamples'], df_plot[score_ref], label=labels[-1], linestyle='--', color='black')
    #ax.legend(loc=loc, fontsize=FONTSIZE, ncol=2)
    ax.set_xlabel("#samples", fontsize=FONTSIZE)
    ax.set_ylim(ylim)
    ax.set_title("{}, {}".format(ylabel,title), fontsize=FONTSIZE)
    ax.tick_params(axis='both', which='major', labelsize=FONTSIZE)
    plt.savefig(filename, dpi=DPI, bbox_inches='tight')
    #!convert -trim $filename $filename
    !pdfcrop $filename $filename

In [ ]:
ti_selection_plot('A', 'a)','mean quadratic score', 'mean_test_brier', FIG_DIR+'ti_selection_A_brier.pdf', 'ref_brier')

In [ ]:
ti_selection_plot('B', 'b)','mean quadratic score', 'mean_test_brier', FIG_DIR+'ti_selection_B_brier.pdf', score_ref='ref_brier')

In [ ]:
ti_selection_plot('C', 'c)','mean quadratic score', 'mean_test_brier', FIG_DIR+'ti_selection_C_brier.pdf', score_ref='ref_brier')

In [ ]:
ti_selection_plot('A', 'a)','mean zero-one score', 'mean_test_zero_one', FIG_DIR+'ti_selection_A_zero_one.pdf', ylim=[0.5,1], score_ref='ref_zero_one', loc="upper left")

In [ ]:
ti_selection_plot('B', 'b)','mean zero-one score', 'mean_test_zero_one', FIG_DIR+'ti_selection_B_zero_one.pdf', ylim=[0.5,1], score_ref='ref_zero_one', loc="upper left")

In [ ]:
ti_selection_plot('C', 'c)','mean zero-one score', 'mean_test_zero_one', FIG_DIR+'ti_selection_C_zero_one.pdf', ylim=[0.5,1], score_ref='ref_zero_one')

In [ ]:
ti_selection_plot('A', 'a)','mean balanced linear score', 'mean_test_linear', FIG_DIR+'ti_selection_A_linear.pdf', ylim=[0.3,1], score_ref='ref_linear')

In [ ]:
ti_selection_plot('B', 'b)','mean balanced linear score', 'mean_test_linear', FIG_DIR+'ti_selection_B_linear.pdf', ylim=[0.3,1], score_ref='ref_linear')

In [ ]:
ti_selection_plot('C', 'c)','mean balanced linear score', 'mean_test_linear', FIG_DIR+'ti_selection_C_linear.pdf', ylim=[0.3,1], score_ref='ref_linear')

## Roussillon

Plot example simulations.

In [ ]:
ti_true = readImageGslib(DATA_DIR+'trueTI.gslib')
ti_analog = readImageGslib(DATA_DIR+'analogTI.gslib')
mask = readImageGslib(DATA_DIR+'mask.gslib')
trend = readImageGslib(DATA_DIR+'trend.gslib')
im_angle = readImageGslib(DATA_DIR+'orientation.gslib')
nx, ny, nz = mask.nx, mask.ny, mask.nz      # number of cells
sx, sy, sz = mask.sx, mask.sy, mask.sz      # cell unit
ox, oy, oz = mask.ox, mask.oy, mask.oz      # origin (corner of the "first" grid cell)

deesse = DeesseEstimator(
    varnames=['X','Y','Z','Facies'],
    nx=nx, ny=ny, nz=nz,
    sx=sx, sy=sy, sz=sz,
    ox=ox, oy=oy, oz=oz,
    nv=2, varname=['Facies', 'trend'],
    nTI=1, TI=ti_true,
    mask=mask.val,
    rotationUsage=1,            # use rotation without tolerance
    rotationAzimuthLocal=True,  #    rotation according to azimuth: local
    rotationAzimuth=im_angle.val[0,:,:,:],      #    rotation azimuth: map of values
    dataImage=trend,
    outputVarFlag=[True, False],
    distanceType=[0,1],
    nneighboringNode=[50,1],
    distanceThreshold=[0.05, 0.05],
    maxScanFraction=0.5,
    npostProcessingPathMax=1,
    seed=20191201,
    nrealization=1,
    nthreads=8,
)

def simulate(nsamples, TI, nneighboringNode,distanceThreshold, maxScanFraction):
    hd = pd.DataFrame(readPointSetGslib(SAMPLES_DIR+'roussillon_observations_{}.gslib'.format(nsamples)).to_dict())
    deesse.set_params(
        TI=TI,
        nneighboringNode=nneighboringNode,
        distanceThreshold=distanceThreshold,
        maxScanFraction=maxScanFraction
    )
    deesse.fit(hd[['X', 'Y', 'Z']], hd['Facies_real00000'])
    return deesse.simulate()

In [ ]:
def plot_example_roussillon(score_name, nsamples, n, t, f, ti_name, removeColorbar=True):

    ti = readImageGslib(ti_name)
    if ti_name == 'data/trueTI.gslib':
        ti_shortname = 'true'
    else:
        ti_shortname = 'analog'
        
    if score_name == 'brier':
        score_name = 'quadratic'
    elif score_name == 'zero_one':
        score_name = 'zero-one'
    filename = FIG_DIR + "ex_roussillon_{0}_{1}_{2}.pdf".format(nsamples, score_name, ti_shortname)


    FONT_SIZE = 16
    COLOR_SCHEME_ROUSSILLON = [ 
            [x/255 for x in [166,206,227]],
            [x/255 for x in [178,223,138]],   
            [x/255 for x in [31,120,180]],
            [x/255 for x in [51,160,44]],
          ]
    LEGEND = ['alluvial fan', 'flood plain', 'splay', 'river bed']
    EXCLUDED_VAL = -9999999

    image = simulate(nsamples, ti, n, t, f)['sim'][0]

    fig = plt.figure(figsize=(5,5))
    fig.subplots_adjust(left=0.05, right=0.9)
    xmin, xmax = [int(x) for x in [image.xmin(), image.xmax()]]
    ymin, ymax = [int(y) for y in [image.ymin(), image.ymax()]]
    drawImage2D(image, excludedVal=EXCLUDED_VAL,
                title = "{0} wells, {1}, {2} TI".format(nsamples, score_name, ti_shortname),
                removeColorbar=removeColorbar,
                categ=True,
                categColbad='white',
                categCol=COLOR_SCHEME_ROUSSILLON,
                cticklabels=LEGEND,
                title_fontsize=FONT_SIZE,
                cticklabels_fontsize=FONT_SIZE,
                xlabels_fontsize=FONT_SIZE,
                ylabels_fontsize=FONT_SIZE,
                xticklabels = [xmin, xmax],
                yticklabels = [ymin, ymax],
                xticklabels_fontsize=FONT_SIZE,
                yticklabels_fontsize=FONT_SIZE,
                xticks=[xmin, xmax],
                yticks=[ymin, ymax],
                ylabel_rotation=0,
               )
    
    #plt.scatter(point_set_roussillon.x(), point_set_roussillon.y(), marker= 'x', s=30, c='black')
    plt.savefig(filename, dpi=DPI, bbox_inches="tight")
    #!convert -trim $filename $filename
    !pdfcrop $filename $filename

In [ ]:
plot_example_roussillon('brier', 150, [10, 1], [0.5, 0.1], 0.001, 'data/trueTI.gslib')

In [ ]:
for index, row in df_best_roussillon.iterrows():
    plot_example_roussillon(row['score_method'], row['nsamples'], eval(row['param_nneighboringNode']),
                           eval(row['param_distanceThreshold']), row['param_maxScanFraction'],
                           row['param_TI'])